# Toy problem for debugging Dual Decomposition

$$
\begin{align*}
\max_{x_k,y} \quad & \sum_k f_k'y + \sum_k c_k'x_k \\
\mathrm{s.t.} \quad & A_k x_k + B_k y \leq d_k, \\
& y\in \{0,1\}.
\end{align*}
$$

$$
\begin{align*}
\max_{x_k,y_k} \quad & \sum_k f_k'y + \sum_k c_k'x_k \\
\mathrm{s.t.} \quad & A_k x_k + B_k y_k \leq d_k, \\
& \sum_k H_k y_k = 0, \\
& y_k\in \{0,1\}.
\end{align*}
$$

In [1]:
CUT_STRATEGY = 'maximal'

In [2]:
from __future__ import division
from cobra import Model
from dynamicme.optimize import Variable, Constraint
from six import iteritems
import numpy as np

### Example from
http://www.optirisk-systems.com/manuals/FortspManual.pdf

$$
\begin{align*}
\max_{x,y} \quad & \sum_k P_k \left( \sum_c p^s_c x^s_{c,k} - p^b_c x^b_{c,k}  - \sum_c c_c a_c \right)\\
\mathrm{s.t.} \quad & \sum_c a_c \leq A \\
& Y_{c,k}  a_c - x^s_{c,k} + x^b_{c,k} \geq R_c \\
& x^s_{\mathrm{beet},k} \leq Q(\mathrm{beet}) \\
& x^s_{\mathrm{beet},c} + s^x_k \leq Y_{\mathrm{beet},k} a_\mathrm{beet}
\end{align*}
$$

In [3]:
from dynamicme.decomposition import LagrangeMaster, LagrangeSubmodel

In [4]:
#------------------------------------------------------------
# Data
crops = ['wheat','corn','beet']
total_area = 500.  # Total acres
prob_dict = {'below':1./3, 'average':1./3, 'above':1./3}
# Yields (tons/acre) over scenarios
yield_dict = {}
yield_dict['below'] = {'wheat':2., 'corn':2.4, 'beet':16.}
yield_dict['average'] = {'wheat':2.5, 'corn':3., 'beet':20.}
yield_dict['above'] = {'wheat':3., 'corn':3.6, 'beet':24.}
# Planting costs
plantcost_dict={'wheat':150, 'corn':230, 'beet':260}   # $/ton
sellprice_dict = {'wheat':170, 'corn':150, 'beet':36}  # $/ton
excess_sell_price = 10    # beet sell price ($/ton) when quota exceeded
beet_quota = 6000   # tons
buyprice_dict = {'wheat':238, 'corn':210, 'beet':100}  # $/ton
req_dict = {'wheat':200., 'corn':240., 'beet':0.} # tons required to feed cattle
#------------------------------------------------------------
# Area is complicating (first-stage) variable

In [5]:
# Create subproblems
UB = 1e4
sub_dict = {}
for scen,yields in iteritems(yield_dict):
    # Each subproblem gets its own copy of first-stage variables
    area_dict = {c:Variable('area_%s'%c, lower_bound=0., upper_bound=1e6,
        objective_coefficient=prob_dict[scen]*plantcost_dict[c]) for c in crops}
    for y in area_dict.values():
        y.variable_kind = 'integer'
    mdl = Model('sub')
    mdl.add_reactions(area_dict.values())
    # Global constraint: sum_a <= Area
    cons_area = Constraint('total_area')
    cons_area._bound = total_area
    cons_area._constraint_sense = 'L'
    mdl.add_metabolites(cons_area)
    for y in area_dict.values():
        y.add_metabolites({cons_area:1.}, combine=False)

    # MAX sell price
    x_excess = Variable('sell_excess_beet', lower_bound=0., upper_bound=UB)
    x_excess.objective_coefficient = -prob_dict[scen]*excess_sell_price    
    mdl.add_reaction(x_excess)
    
    for c in crops:
        ### Scenario variables x: tons sold, bought, excess sold per scenario
        x_sell = Variable('sell_%s'%(c), lower_bound=0., upper_bound=UB)
        x_buy  =  Variable('buy_%s'%(c), lower_bound=0., upper_bound=UB)
        mdl.add_reactions([x_sell,x_buy])        
        x_sell.objective_coefficient = -prob_dict[scen]*sellprice_dict[c]
        x_buy.objective_coefficient = prob_dict[scen]*buyprice_dict[c]
        
        ### Scenario constraint: Y_ck*ac - xs_ck + xb_ck >= Req_c        
        cons = Constraint('required_%s'%c)
        cons._bound = req_dict[c]
        cons._constraint_sense = 'G'        
        mdl.add_metabolites(cons)
        area = area_dict[c]
        area.add_metabolites({cons:yield_dict[scen][c]})
        x_sell.add_metabolites({cons:-1.})
        x_buy.add_metabolites({cons:1.})
        
        ### Beet quota
        if c=='beet':
            x_sell.upper_bound = beet_quota
            # xs_beet,k + xexcess_k <= Yield_beet,k * area_beet
            cons_beet = Constraint('beet_balance')
            cons_beet._bound = 0.
            cons_beet._constraint_sense = 'L'
            x_sell.add_metabolites({cons_beet:1.})
            x_excess.add_metabolites({cons_beet:1.})
            area_dict[c].add_metabolites({cons_beet:-yield_dict[scen][c]})
    
    sub = LagrangeSubmodel(mdl, scen)
    sub_dict[scen] = sub

In [6]:
master = LagrangeMaster(mdl)
master._INF = 1e6
master.add_submodels(sub_dict)
master._z.LB = -master._INF
master._z.UB = master._INF
master.model.update()

In [7]:
for v in master.model.getVars():
    print(v.VarName, v.LB, v.UB)

('z', -1000000.0, 1000000.0)
('tk_below', -1000000.0, 1000000.0)
('tk_average', -1000000.0, 1000000.0)
('tk_above', -1000000.0, 1000000.0)
('u_00', -1000000.0, 1000000.0)
('u_01', -1000000.0, 1000000.0)
('u_10', -1000000.0, 1000000.0)
('u_11', -1000000.0, 1000000.0)
('u_20', -1000000.0, 1000000.0)
('u_21', -1000000.0, 1000000.0)


In [8]:
master._us

[<gurobi.Var u_00>,
 <gurobi.Var u_01>,
 <gurobi.Var u_10>,
 <gurobi.Var u_11>,
 <gurobi.Var u_20>,
 <gurobi.Var u_21>]

In [9]:
for sub_ind,sub in iteritems(sub_dict):
    print('H_%s'%sub_ind)
    print(sub._H.todense())

H_below
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
H_average
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]
H_above
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]


In [10]:
cc = master.model.getConstrs()[0]
print(master.model.getRow(cc), cc.Sense, cc.RHS)

(<gurobi.LinExpr: z + -1.0 tk_below + -1.0 tk_average + -1.0 tk_above>, '<', 0.0)


In [11]:
master.model.getObjective()

<gurobi.QuadExpr: z + [ -0.5 u_00 ^ 2 + -0.5 u_01 ^ 2 + -0.5 u_10 ^ 2 + -0.5 u_11 ^ 2 + -0.5 u_20 ^ 2 + -0.5 u_21 ^ 2 ]>

In [12]:
cz = master.model.getConstrByName('z_cut')
print(master.model.getRow(cz), cz.Sense, cz.RHS)

(<gurobi.LinExpr: z + -1.0 tk_below + -1.0 tk_average + -1.0 tk_above>, '<', 0.0)


In [13]:
from dynamicme.callback_gurobi import cb_benders_multi

# Solve using callbacks

In [14]:
master.model.Params.Presolve = 0
master.print_iter=1
uopt = master.optimize() 

        Iter          UB          LB     Best UB     Best LB         gap   relgap(%)     time(s)
           0       1e+06  -1.154e+05       1e+06  -1.154e+05   1.115e+06       111.5    0.007625
           1  -9.971e+04  -1.413e+05  -9.971e+04  -1.154e+05   1.569e+04       15.73    0.010076
           2   -1.07e+05  -1.142e+05   -1.07e+05  -1.142e+05        7185       6.715    0.012454
           3  -1.075e+05  -1.104e+05  -1.075e+05  -1.104e+05        2880       2.679    0.014895
           4  -1.095e+05  -1.095e+05  -1.095e+05  -1.095e+05   1.081e-07   9.871e-11    0.017168
relgap (%g) <= gaptol (%g). Finished.


## Check answer: should be 
- {'wheat':170, 'corn':80, 'beet':250}
- objval = (-)108390

In [15]:
uopt

array([ 17.        ,   3.        , -20.        ,  50.        ,
       -16.66666667, -33.33333333])

In [16]:
master._z

<gurobi.Var z (value -109506.666667)>

In [17]:
x_dict = {}
for scen,sub in iteritems(master.sub_dict):
    #sub.update_obj(yopt)
    #sub.model.optimize()
    x_dict[scen] = {v.VarName:v.X for v in sub.model.getVars()}

In [18]:
x_dict

{'above': {'area_beet': 250.0,
  'area_corn': 183.0,
  'area_wheat': 67.0,
  'buy_beet': 0.0,
  'buy_corn': 0.0,
  'buy_wheat': 0.0,
  'sell_beet': 6000.0,
  'sell_corn': 418.8000000000005,
  'sell_excess_beet': 0.0,
  'sell_wheat': 0.9999999999999858},
 'average': {'area_beet': 300.0,
  'area_corn': 80.0,
  'area_wheat': 120.0,
  'buy_beet': 0.0,
  'buy_corn': 0.0,
  'buy_wheat': 0.0,
  'sell_beet': 6000.0,
  'sell_corn': 0.0,
  'sell_excess_beet': 0.0,
  'sell_wheat': 100.0},
 'below': {'area_beet': 300.0,
  'area_corn': 100.0,
  'area_wheat': 100.0,
  'buy_beet': 0.0,
  'buy_corn': 0.0,
  'buy_wheat': 0.0,
  'sell_beet': 4800.0,
  'sell_corn': 0.0,
  'sell_excess_beet': 0.0,
  'sell_wheat': 0.0}}

In [21]:
sub._ys

[<gurobi.Var area_beet (value 250.0)>,
 <gurobi.Var area_corn (value 183.0)>,
 <gurobi.Var area_wheat (value 67.0)>]

In [41]:
Hys = []
for sub_ind,sub in iteritems(sub_dict):
    yk = np.array([x.X for x in sub._ys])
    Hyk = sub._H*yk
    Hys.append(Hyk)    
    print('Scenario: %s'%sub_ind)
    print('yk = %s'%yk)
    print('Hk*yk =%s' % Hyk)
    
print("\nsum_k Hk*yk=")
print(sum(Hys))

Scenario: below
yk = [300. 100. 100.]
Hk*yk =[300. 100. 100.   0.   0.   0.]
Scenario: average
yk = [300.  80. 120.]
Hk*yk =[-300.  -80. -120.  300.   80.  120.]
Scenario: above
yk = [250. 183.  67.]
Hk*yk =[   0.    0.    0. -250. -183.  -67.]

sum_k Hk*yk=
[   0.   20.  -20.   50. -103.   53.]


## Check that all constraints satisfied

In [37]:
# Total area
for sub_ind,sub in iteritems(sub_dict):
    yopt = [v.X for v in sub._ys]
    print('Scenario=%s. Total area constraint:'%sub_ind, sum(yopt) <= total_area)

('Scenario=below. Total area constraint:', True)
('Scenario=average. Total area constraint:', True)
('Scenario=above. Total area constraint:', True)


In [ ]:
# Cattle feed crop requirement
y_dict = {y.VarName:y.X for y in master._ys}
for scen in prob_dict.keys():
    for c,req in iteritems(req_dict):
        area = y_dict['area_%s'%c]
        xsell = x_dict[scen]['sell_%s'%(c)]
        xbuy = x_dict[scen]['buy_%s'%(c)]
        lhs = yield_dict[scen][c]*area - xsell + xbuy
        rhs = req
        print('%s, %s: %g >= %g, %s'%(scen,c,lhs,rhs,lhs>=rhs))

master.model.optimize(cb_benders_multi)

yopt = np.array([y.X for y in master._ys])
xopt = []
for k in range(len(c)):
    sub = sub_dict[k]
    sub.update_obj(yopt)
    sub.model.optimize()
    xc = sub.model.model.getConstrByName('x%d'%k)
    xopt.append(xc.Pi)